--- 

# <center> Project: NLP ENSAE 
## <center> Intents Classification for Neural Text Generation

<center>Work done by : 

##### <center> Ali HAIDAR email : ali.haidar@polytechnique.edu
##### <center> François Bertholom   email : 

---

In [1]:
import numpy as np 
import pandas as pd
from datasets import load_dataset
from keras_preprocessing.sequence import pad_sequences
from pytorch_pretrained_bert import BertTokenizer
from pytorch_pretrained_bert import BertModel
import torch
from torch import nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils import clip_grad_norm_
from IPython.display import clear_output

import process
import BertMultiClassifier

## Load data 

In [63]:
dataset = load_dataset('silicone','dyda_da')

Found cached dataset silicone (C:/Users/aliha/.cache/huggingface/datasets/silicone/dyda_da/1.0.0/af617406c94e3f78da85f7ea74ebfbd3f297a9665cb54adbae305b03bc4442a5)


  0%|          | 0/3 [00:00<?, ?it/s]

In [64]:
train = pd.DataFrame(data=dataset['train'])
val = pd.DataFrame(data=dataset['validation'])
test = pd.DataFrame(data=dataset['test'])

### Building a classifier based on Bert that takes on input a uterance and gives the label

In [65]:
X_train = train['Utterance']
y_train = np.array(train['Label'])

X_val = val['Utterance']
y_val = np.array(val['Label'])

X_test = test['Utterance']
y_test = np.array(test['Label'])

n_classes = len(np.unique(y_train))


In [69]:
train_tokens_ids = process.tokenize(X_train)
val_tokens_ids = process.tokenize(X_val)
test_tokens_ids = process.tokenize(X_test)

In [70]:
train_masks = process.mask(train_tokens_ids)
val_masks = process.mask(val_tokens_ids)
test_masks = process.mask(test_tokens_ids)

In [ ]:
model_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t.to(device) for t in batch_data)

        logits = model_clf(token_ids, masks)
        loss_func = nn.CrossEntropyLoss()
        loss = loss_func(logits, labels.squeeze())
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])

np.mean(bert_predicted)       
print(classification_report(test_y, bert_predicted))